In [2]:
def concat_inputs(report_dir, files, truth_dir=None, mode='train'):
    
    data = pd.read_csv(os.path.join(report_dir, files[0]+'.csv'), sep=',')[['id','proba']]
    
    for i in range(len(files)-1):
        join = pd.read_csv(os.path.join(report_dir, files[i+1]+'.csv'), sep=',')['proba']
        data = pd.concat([data, join], axis=1)
        
    if mode == 'train':
        truth = pd.read_json(truth_dir, orient='records', lines=True)
        label = truth['label']
        data = pd.concat([data, label], axis=1)

    elif mode == 'test':
        join = data['id'].astype("string").str.cat(['jpg']*data.shape[0], sep='.')
        
    data.drop(columns=['id'], inplace=True)
        
    return data, join

In [3]:
from torch.utils.data import Dataset, DataLoader

class mamiDataset(Dataset):
    
    def __init__(self, data, mode='train'):
        self.mode = mode
        if self.mode == 'train':
            self.inp = data.iloc[:,:-1].values
            self.oup = data.iloc[:,-1].values.reshape(-1,1)
        else:
            self.inp = data.values
            
    def __len__(self):
        return len(self.inp)
    
    def __getitem__(self, idx):
        if self.mode == 'train':
            inpt  = torch.Tensor(self.inp[idx])
            oupt  = torch.Tensor(self.oup[idx])
            return {'inp': inpt,
                    'oup': oupt}
        else:
            inpt = torch.Tensor(self.inp[idx])
            return {'inp': inpt}

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    
    def __init__(self, input_dim=7):
        super().__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(self.input_dim, 16)
        self.b1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.b2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8,2)

    def forward(self,x):
        x = self.fc1(x)
        x = self.b1(torch.sigmoid(x))
        x = self.fc2(x)
        x = self.b2(torch.sigmoid(x))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x
    
# class Network(nn.Module):
    
#     def __init__(self, input_dim=7):
#         super().__init__()
#         self.input_dim = input_dim
#         self.fc1 = nn.Linear(self.input_dim, 8)
#         self.b1 = nn.BatchNorm1d(8)
#         self.fc2 = nn.Linear(8,2)

#     def forward(self,x):
#         x = self.fc1(x)
#         x = self.b1(torch.sigmoid(x))
#         x = self.fc2(x)
#         x = F.softmax(x, dim=1)
#         return x

# F.relu(x)

In [5]:
def train(model, x, y, optimizer, criterion):
    
    model.zero_grad()
    output = model(x)
    y = y.to(torch.int64)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    return loss, output

In [6]:
from sklearn import metrics 
def scoreA(labels, truth):
        
    matrix = metrics.confusion_matrix(truth, labels)

    #positive label
    if matrix[0][0] == 0:
        pos_precision = 0.0
        pos_recall = 0.0
    else:
        pos_precision = matrix[0][0]*1.0 / (matrix[0][0] + matrix[0][1])
        pos_recall = matrix[0][0]*1.0 / (matrix[0][0] + matrix[1][0])

    if (pos_precision + pos_recall) != 0:
        pos_F1 = 2 * (pos_precision * pos_recall)*1.0 / (pos_precision + pos_recall)
    else:
        pos_F1 = 0

    #negative label
    neg_matrix = [[matrix[1][1], matrix[1][0]], [matrix[0][1], matrix[0][0]]]

    if neg_matrix[0][0] == 0:
        neg_precision = 0.0
        neg_recall = 0.0
    else:
        neg_precision = neg_matrix[0][0]*1.0 / (neg_matrix[0][0] + neg_matrix[0][1])
        neg_recall = neg_matrix[0][0]*1.0 / (neg_matrix[0][0] + neg_matrix[1][0])

    if (neg_precision + neg_recall) != 0:
        neg_F1 = 2 * (neg_precision * neg_recall) / (neg_precision + neg_recall)
    else:
        neg_F1 = 0

    f1 = (pos_F1 + neg_F1) / 2
    
    return f1

In [7]:
from torch.optim import Adam
criterion = nn.CrossEntropyLoss()

In [8]:
from tqdm import tqdm

def training_loop(dataloader, net, optm, criterion, device):

    for epoch in range(EPOCHS):
        epoch_loss = 0
        correct = 0
        for bidx, batch in tqdm(enumerate(dataloader)):
            x_train, y_train = batch['inp'], batch['oup']
            y_train = y_train.squeeze(axis=1)
            x_train = x_train.view(-1,INPUT_DIM)
            
            loss, predictions = train(net, x_train, y_train, optm, criterion)
            predictions = predictions.argmax(dim=1)
            
            for idx, i in enumerate(predictions):
                if i == y_train[idx]:
                    correct += 1
            epoch_loss += loss


        print('\033[30m'+'Epoch {} scoreA : {}'.format(epoch+1, scoreA(predictions, y_train)))
        print('\033[30m'+'Epoch {} Loss : {}'.format(epoch+1, epoch_loss))

## Neural network

In [18]:
import os
import torch
import pandas as pd
BATCH_SIZE = 16
EPOCHS = 5
INPUT_DIM = 3
mi_file = ['./save/new_mami/misogynous_concat_bert', './save/new_mami/misogynous_mmf_transformer', './save/new_mami/misogynous_visual_bert']
sh_file = ['./save/new_mami/shaming_late_fusion', './save/new_mami/shaming_mmbt', './save/new_mami/shaming_visual_bert']
st_file = ['./save/new_mami/stereotype_late_fusion', './save/mami/stereotype_mmf_transformer', './save/mami/stereotype_visual_bert']
ob_file = ['./save/new_mami/objectification_concat_bert', './save/new_mami/objectification_mmf_transformer', './save/mami/objectification_vilbert']
vi_file = ['./save/new_mami/violence_concat_bert', './save/mami/violence_mmf_transformer', './save/mami/violence_vilbert']
files = [mi_file, sh_file, st_file, ob_file, vi_file]

In [20]:
truth_list = ['/home/taochen/mmf/data/datasets/mami/defaults/annotations/val_misogynous.jsonl',
             '/home/taochen/mmf/data/datasets/mami/defaults/annotations/val_shaming.jsonl',
             '/home/taochen/mmf/data/datasets/mami/defaults/annotations/val_stereotype.jsonl',
             '/home/taochen/mmf/data/datasets/mami/defaults/annotations/val_objectification.jsonl',
             '/home/taochen/mmf/data/datasets/mami/defaults/annotations/val_violence.jsonl']
col_names = ['misogynous', 'shaming', 'stereotype', 'objectification', 'violence']
nets = []

gpu = 0
device = torch.device(gpu if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(gpu)
    
for idx in range(len(files)):
    truth_dir = truth_list[idx]
    data, comp = concat_inputs(report_dir='', files=files[idx], truth_dir=truth_dir)
    
    data_train = mamiDataset(data=data)
    data_test = torch.tensor(data.iloc[800:,:-1].values).to(torch.float32)
    dataloader = DataLoader(dataset = data_train, batch_size = BATCH_SIZE, shuffle = False)
    
    net = Network(INPUT_DIM)
    optm = Adam(net.parameters(), lr = 0.001)
    training_loop(dataloader, net, optm, criterion, device)
    res = net(data_test).argmax(axis=1)
    
    truth = pd.read_json(truth_dir, orient='records', lines=True)[800:]
    truth = truth['label']
    torch.tensor(truth.values)
    
    comp = comp[800:]
    comp = torch.tensor((comp > 0.5).values).to(torch.int64)
    
    print('\033[34m' + col_names[idx]+ '\t' + str(scoreA(res, truth))+ '\t' + str(scoreA(comp, truth)))
    nets.append(net)

63it [00:00, 533.23it/s]
63it [00:00, 586.43it/s]
0it [00:00, ?it/s]

Epoch 1 scoreA : 0.873015873015873
Epoch 1 Loss : 48.52399444580078
Epoch 2 scoreA : 0.7499999999999999
Epoch 2 Loss : 32.802555084228516


63it [00:00, 603.34it/s]
63it [00:00, 603.65it/s]
0it [00:00, ?it/s]

Epoch 3 scoreA : 0.7499999999999999
Epoch 3 Loss : 30.8625431060791
Epoch 4 scoreA : 0.7499999999999999
Epoch 4 Loss : 30.33580780029297


63it [00:00, 602.33it/s]
63it [00:00, 602.11it/s]
0it [00:00, ?it/s]

...

Epoch 1 scoreA : 0.6666666666666666
Epoch 1 Loss : 40.74103927612305
Epoch 2 scoreA : 0.6666666666666666
Epoch 2 Loss : 35.11079788208008


63it [00:00, 604.04it/s]
63it [00:00, 605.24it/s]
0it [00:00, ?it/s]

Epoch 3 scoreA : 0.7948717948717948
Epoch 3 Loss : 32.44279098510742
Epoch 4 scoreA : 0.7948717948717948
Epoch 4 Loss : 30.688753128051758


63it [00:00, 602.52it/s]

Epoch 5 scoreA : 1.0
Epoch 5 Loss : 29.453937530517578
violence	0.705232129697863	0.5268817204301076


In [21]:
mi_file = ['./save/new_mami_submission/misogynous_concat_bert', './save/new_mami_submission/misogynous_mmf_transformer', './save/new_mami_submission/misogynous_visual_bert']
sh_file = ['./save/new_mami_submission/shaming_late_fusion', './save/new_mami_submission/shaming_mmbt', './save/new_mami_submission/shaming_visual_bert']
st_file = ['./save/new_mami_submission/stereotype_late_fusion', './save/mami_submission/stereotype_mmf_transformer', './save/mami_submission/stereotype_visual_bert']
ob_file = ['./save/new_mami_submission/objectification_concat_bert', './save/new_mami_submission/objectification_mmf_transformer', './save/mami_submission/objectification_vilbert']
vi_file = ['./save/new_mami_submission/violence_concat_bert', './save/mami_submission/violence_mmf_transformer', './save/mami_submission/violence_vilbert']
files = [mi_file, sh_file, st_file, ob_file, vi_file]

for idx in range(len(files)):
    
    data, ids = concat_inputs(report_dir='', files=files[idx], mode='test')
        
    net = nets[idx]
    logits = net(torch.tensor(data.values, dtype=torch.float32))
    res = logits.argmax(axis=1).to(torch.int64).unsqueeze(axis=1)

    if idx == 0:
        submission = res
    else:
        submission = torch.cat((submission, res), dim=1)

out_path = 'answer_nn.txt'
submission = pd.DataFrame(submission.numpy())
submission = pd.concat([pd.DataFrame(ids), submission], axis=1)

# # fix misogynous columns
# temp = submission[submission.iloc[:,1] == 0]
# temp = temp.iloc[:,2:].sum(axis=1) > 1
# temp = temp.loc[temp == True].index.to_list()
# submission.iloc[temp,1] = 1

# # fix other four columns
# filter_ = submission.iloc[:,1] == 0
# submission[filter_] = submission[filter_].replace(1, 0)
submission.to_csv(out_path, header=False, index=False, sep='\t')

## Average

In [130]:
mi_file = ['./save/new_mami_submission/misogynous_concat_bert', './save/new_mami_submission/misogynous_mmf_transformer', './save/new_mami_submission/misogynous_visual_bert']
sh_file = ['./save/new_mami_submission/shaming_late_fusion', './save/new_mami_submission/shaming_mmbt', './save/new_mami_submission/shaming_visual_bert']
st_file = ['./save/new_mami_submission/stereotype_late_fusion', './save/mami_submission/stereotype_mmf_transformer', './save/mami_submission/stereotype_visual_bert']
ob_file = ['./save/new_mami_submission/objectification_concat_bert', './save/new_mami_submission/objectification_mmf_transformer', './save/mami_submission/objectification_vilbert']
vi_file = ['./save/new_mami_submission/violence_concat_bert', './save/mami_submission/violence_mmf_transformer', './save/mami_submission/violence_vilbert']
files = [mi_file, sh_file, st_file, ob_file, vi_file]

for idx in range(len(files)):
    
    data, ids = concat_inputs(report_dir='', files=files[idx], mode='test') 
    res = pd.DataFrame(((data.sum(axis=1)/len(data.columns)) > 0.5).astype(int))
    
    if idx == 0:
        submission = res
    else:
        submission = pd.concat([submission, res], axis=1)

out_path = 'answer_ave.txt'
submission = pd.concat([pd.DataFrame(ids), submission], axis=1)

# # fix misogynous columns
# temp = submission[submission.iloc[:,1] == 0]
# temp = temp.iloc[:,2:].sum(axis=1) > 1
# temp = temp.loc[temp == True].index.to_list()
# submission.iloc[temp,1] = 1

# # fix other four columns
# filter_ = submission.iloc[:,1] == 0
# submission[filter_] = submission[filter_].replace(1, 0)
submission.to_csv(out_path, header=False, index=False, sep='\t')

# Final averaging

In [25]:
ave = 'answer_ave.txt'
nn = 'answer_nn.txt'
mul = 'answer_mul.txt'
ave = pd.read_csv(ave, sep='\t')
nn = pd.read_csv(nn, sep='\t')
mul = pd.read_csv(mul, sep='\t')
ave

,img,mi,sh,st,ob,vi
0,15236.jpg,1,0,0,0,0
1,15805.jpg,1,0,1,0,0
2,16254.jpg,1,0,0,0,0
3,16191.jpg,0,0,0,1,0
4,15952.jpg,1,0,0,0,0
...,...,...,...,...,...,...
995,15591.jpg,1,1,0,1,0
996,15049.jpg,1,1,0,0,0
997,15363.jpg,1,1,1,1,0
998,15199.jpg,1,0,0,0,0


In [26]:
columns = ave.columns.to_list()
columns.remove('img')
for col in columns:
    ave[col] = ((ave[col] + nn[col] + mul[col]) / 3 > 0.5).astype(int)

temp = ave[ave.iloc[:,1] == 0]
temp = temp.iloc[:,2:].sum(axis=1) > 1
temp = temp.loc[temp == True].index.to_list()
ave.iloc[temp,1] = 1

filter_ = ave.iloc[:,1] == 0
ave[filter_] = ave[filter_].replace(1, 0)
ave.to_csv('answer.txt', header=False, index=False, sep='\t')

In [27]:
ave

,img,mi,sh,st,ob,vi
0,15236.jpg,1,0,0,0,0
1,15805.jpg,1,1,1,0,0
2,16254.jpg,1,0,0,0,0
3,16191.jpg,0,0,0,0,0
4,15952.jpg,1,0,0,0,0
...,...,...,...,...,...,...
995,15591.jpg,1,1,1,1,0
996,15049.jpg,1,1,0,0,0
997,15363.jpg,1,1,1,1,0
998,15199.jpg,1,0,0,0,0
